In [1]:
import sys
sys.path.append('/home/lessa/smodels')
from smodels.experiment.databaseObj import Database
from smodels.tools.physicsUnits import GeV,fb,pb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from smodels.tools.databaseBrowser import Browser
import numpy as np

In [2]:
db = Database('/home/lessa/smodels-database')

In [7]:
exp = db.getExpResults(analysisIDs=['ATLAS-CONF-2013-062'],dataTypes=['efficiencyMap'],txnames=['T5bbbb'],
                       useSuperseded=True,useNonValidated=True,datasetIDs=['SL2b_low'], discard_zeroes = True)[0]

WARNING in databaseObj.getExpResults() in 501: validated is None in ATLAS-CONF-2013-062/SL2b_low/T1. Please set to True, False, N/A, or tbd.
WARNING in databaseObj.getExpResults() in 501: validated is None in ATLAS-CONF-2013-062/SL2b_low/T1bbbb. Please set to True, False, N/A, or tbd.
WARNING in databaseObj.getExpResults() in 501: validated is None in ATLAS-CONF-2013-062/SL2b_low/T1bbbt. Please set to True, False, N/A, or tbd.
WARNING in databaseObj.getExpResults() in 501: validated is None in ATLAS-CONF-2013-062/SL2b_low/T1bbqq. Please set to True, False, N/A, or tbd.
WARNING in databaseObj.getExpResults() in 501: validated is None in ATLAS-CONF-2013-062/SL2b_low/T1bbtt. Please set to True, False, N/A, or tbd.
WARNING in databaseObj.getExpResults() in 501: validated is None in ATLAS-CONF-2013-062/SL2b_low/T1btbt. Please set to True, False, N/A, or tbd.
WARNING in databaseObj.getExpResults() in 501: validated is None in ATLAS-CONF-2013-062/SL2b_low/T1btqq. Please set to True, False, N/

In [8]:
tx = exp.getTxNames()[0]
print tx,tx.constraint

T5bbbb [[[b],[b]],[[b],[b]]]


In [9]:
m = [[1.14888763E+03*GeV, 1.13814300E+03*GeV, 1.09308900*GeV]]*2
porig = tx.txnameData.flattenMassArray(m)
print 'porig=',porig
pt = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
print pt
pt = np.dot(pt,tx.txnameData._V)  ## rotate
print pt
pt = np.array(pt[:tx.txnameData.dimensionality])
print 'Pt=',pt,'val=',tx.txnameData.getValueFor(m)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ptm = [pt[0],pt[1],tx.txnameData.getValueFor(m).asNumber(pb)]
# ax.scatter([ptm[0]], [ptm[1]], [ptm[2]],c='r',marker='o')
xpts = []
ypts = []
zpts = []
for ipt in tx.txnameData.tri.simplices[tx.txnameData.tri.find_simplex(pt)]:
    xpts.append(tx.txnameData.tri.points[ipt][0])
    ypts.append(tx.txnameData.tri.points[ipt][1])
    if tx.txnameData.dimensionality == 3:
        zpts.append(tx.txnameData.tri.points[ipt][2])
    else:
        zpts.append(tx.txnameData.xsec[ipt])
    print tx.txnameData.tri.points[ipt],tx.txnameData.xsec[ipt]

porig= [1148.88763, 1138.143, 1.093089, 1148.88763, 1138.143, 1.093089]
[-281.11237000000006, 375.02787584650116, -374.0558951986456, -281.11237000000006, 375.02787584650116, -374.0558951986456]
[ -6.90753232e+01  -3.39735096e+02   7.73940847e+02   1.92157401e-12
  -1.08357767e-12  -5.68434189e-13]
Pt= [ -69.07532316 -339.73509623  773.94084661] val= 0.000310550191845
[-833.8938036   632.9895     -142.14081683] 0.0
[-356.52075002  676.69941316   69.96861532] 0.0
[-674.5160104   864.80850388 -171.44386233] 0.0
[-671.54461079  862.03429512 -177.22958183] 0.0


In [ ]:
for pt in tx.txnameData._data:
    print pt[0][0],pt[1]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xpts, ypts, zpts,s=80,c='r',marker='o')
ax.scatter([pt[0]],[pt[1]],[pt[2]],s=80,c='b',marker='o')
ax.set_xlabel('x')
ax.set_ylabel('y')
if tx.txnameData.dimensionality == 3:
    ax.set_zlabel('UL')
else:
    ax.set_zlabel('z')

plt.show()

In [ ]:
uvw = [pt[:3]]
fill_value=np.nan
tol = sys.float_info.epsilon * 1e06
simplex = tx.txnameData.tri.find_simplex(uvw, tol=tol)
vertices = np.take(tx.txnameData.tri.simplices, simplex, axis=0)
temp = np.take(tx.txnameData.tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = uvw - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
tx.txnameData.vtx = vertices
tx.txnameData.wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
if type(tx.txnameData.xsec[0]) == float:
    values = np.array ( [ float(x) for x in tx.txnameData.xsec ] )
else:
    values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
print 'v=',tx.txnameData.wts
ret = np.einsum('nj,nj->n', np.take(values, tx.txnameData.vtx), tx.txnameData.wts)
print 'r=',ret
with np.errstate(invalid='ignore'):
    ret[np.any(tx.txnameData.wts < -1e-10, axis=1)] = fill_value
print ret

In [ ]:
fig.clear()
plt.triplot(xpts,ypts)
plt.plot([ptm[0]], [ptm[1]],'ro',xpts,ypts,'bo')
plt.show()

In [ ]:
tri = tx.txnameData.tri
simplex = tri.find_simplex([pt])
vertices = np.take(tri.simplices, simplex, axis=0)
temp = np.take(tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = [pt] - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
vtx = vertices
wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
ret = np.einsum('nj,nj->n', np.take(values, vtx), wts)
with np.errstate(invalid='ignore'):
    ret[np.any(wts < -1e-10, axis=1)] = np.nan
print ret[0]

In [ ]:
vertices

In [ ]:
pt

In [ ]:
m = [[347.406576078*GeV,166.767198233*GeV],[347.406576078*GeV,166.767198233*GeV]]
porig = tx.txnameData.flattenMassArray(m)
ptA = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
ptA = np.dot(ptA,tx.txnameData._V)  ## rotate
print ptA,tx.txnameData.countNonZeros(ptA)

In [ ]:
print tx.txnameData.getValueFor(m)